# Road Accident Risk Prediction (Kaggle Top 20%)

**Author:** Mohammad Akifullah
**Date:** November 8, 2025

### Project Summary
This project tackles a Kaggle regression competition to predict a continuous `accident_risk` score (from 0 to 1). The workflow includes:
1.  **Data Loading & Inspection:** Understanding the dataset.
2.  **Feature Engineering:** Experimenting with feature crosses and interactions.
3.  **Baseline Modeling:** Training and evaluating `XGBoost` and `LightGBM`.
4.  **Advanced Ensembling:** Testing both simple averaging and stacking to combine model predictions.
5.  **Hyperparameter Optimization:** Using Bayesian Optimization to find the best parameters for our models.
6.  **Final Submission:** Creating the submission file based on the best-performing strategy.

This notebook achieved a final leaderboard score of **0.05581**, placing **770 out of 4083**.

### 1. Setup and Imports
First, we'll install `catboost` and import all the necessary libraries for data manipulation, visualization, modeling, and evaluation.

In [3]:
!pip install catboost bayesian-optimization

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing and Metrics
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler

# Models
from sklearn import linear_model
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

# Hyperparameter Tuning
from bayes_opt import BayesianOptimization


### 2. Data Loading and Initial Inspection
We load the `train.csv` dataset and perform an initial review using `.head()`, `.info()`, and `.describe()` to understand its structure, data types, and statistical properties.

In [5]:
df = pd.read_csv('data/train.csv')
df.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [6]:
df.describe()

,id,num_lanes,curvature,speed_limit,num_reported_accidents,accident_risk
count,517754.000000,517754.000000,517754.000000,517754.000000,517754.000000,517754.000000
mean,258876.500000,2.491511,0.488719,46.112575,1.187970,0.352377
std,149462.849974,1.120434,0.272563,15.788521,0.895961,0.166417
min,0.000000,1.000000,0.000000,25.000000,0.000000,0.000000
25%,129438.250000,1.000000,0.260000,35.000000,1.000000,0.230000
50%,258876.500000,2.000000,0.510000,45.000000,1.000000,0.340000
75%,388314.750000,3.000000,0.710000,60.000000,2.000000,0.460000
max,517753.000000,4.000000,1.000000,70.000000,7.000000,1.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517754 entries, 0 to 517753
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      517754 non-null  int64  
 1   road_type               517754 non-null  object 
 2   num_lanes               517754 non-null  int64  
 3   curvature               517754 non-null  float64
 4   speed_limit             517754 non-null  int64  
 5   lighting                517754 non-null  object 
 6   weather                 517754 non-null  object 
 7   road_signs_present      517754 non-null  bool   
 8   public_road             517754 non-null  bool   
 9   time_of_day             517754 non-null  object 
 10  holiday                 517754 non-null  bool   
 11  school_season           517754 non-null  bool   
 12  num_reported_accidents  517754 non-null  int64  
 13  accident_risk           517754 non-null  float64
dtypes: bool(4), float64(

### 3. Feature Engineering and Preprocessing
This is a critical step where we convert categorical data and engineer new features based on domain knowledge.

First, we one-hot encode the primary categorical columns (`lighting`, `weather`, `time_of_day`, `road_type`).

In [8]:
df = pd.get_dummies(df, columns=['lighting', 'weather', 'time_of_day', 'road_type'])

In [9]:
# df['speed_per_lane'] = df['speed_limit'] / df['num_lanes']

In [10]:
# df['speed_limit_known'] = round(df['road_signs_present'] / df['speed_limit'])

In [11]:
# df['weather_time'] = df['weather'].astype(str) + '_' + df['time_of_day'].astype(str)

# df['road_type_time'] = df['road_type'].astype(str) + '_' + df['time_of_day'].astype(str)

# df['road_type_weather'] = df['road_type'].astype(str) + '_' + df['weather'].astype(str)

# df['weather_holiday'] = df['weather'].astype(str) + '_' + df['holiday'].astype(str)

# df['school_season_holiday'] = df['school_season'].astype(str) + '_' + df['holiday'].astype(str)

# df['changing_limit'] = df['speed_limit'].astype(str) + '_' + df['weather'].astype(str)

In [12]:
# df["cornering_speed"] = df['curvature'] * df['speed_limit'] / df['num_lanes']

In [13]:
# df['road_safety'] = df['road_signs_present'] * df['speed_limit']

Next, we engineer interaction features. Based on the hypothesis that road curvature is more dangerous on roads with fewer lanes, we create `curvature_lane_interaction`. Other experimental features (commented out) were tested but did not provide a significant score improvement and were excluded from the final model.

In [14]:
df['curvature_lane_interaction'] = df['curvature'] / df['num_lanes'] #0.8857417590970313

# df['no_of_lanes_safe'] = df['num_lanes'].replace(0, 1) # Or handle appropriately
# # df['curvature_lane_interaction'] = df['road_curvature'] / df['no_of_lanes_safe']

# df['curvature_lane_interaction'] = df['curvature'] * (1 / df['no_of_lanes_safe']) #0.8857417590970313

### 4. Feature and Target Definition
We check the correlation of our new features with the target variable, `accident_risk`, and then define our final feature matrix `X` and target vector `y`. We drop the `id` column and non-predictive features identified during EDA (`road_signs_present`, `school_season`).

In [15]:
series= df.corr()
series['accident_risk'].sort_values(ascending=False)

,accident_risk
accident_risk,1.000000
curvature,0.543946
lighting_night,0.465798
speed_limit,0.430898
curvature_lane_interaction,0.358204
num_reported_accidents,0.213891
weather_foggy,0.149758
holiday,0.051129
weather_rainy,0.036137
public_road,0.031032


In [16]:
y = df['accident_risk']
X = df.drop(['accident_risk', 'id', 'road_signs_present', 'school_season'], axis=1)

In [17]:
X.head()

,num_lanes,curvature,speed_limit,public_road,holiday,num_reported_accidents,lighting_daylight,lighting_dim,lighting_night,weather_clear,weather_foggy,weather_rainy,time_of_day_afternoon,time_of_day_evening,time_of_day_morning,road_type_highway,road_type_rural,road_type_urban,curvature_lane_interaction
0,2,0.06,35,True,False,1,True,False,False,False,False,True,True,False,False,False,False,True,0.0300
1,4,0.99,35,False,True,0,True,False,False,True,False,False,False,True,False,False,False,True,0.2475
2,4,0.63,70,True,True,2,False,True,False,True,False,False,False,False,True,False,True,False,0.1575
3,4,0.07,35,True,False,1,False,True,False,False,False,True,False,False,True,True,False,False,0.0175
4,1,0.58,60,False,True,1,True,False,False,False,True,False,False,True,False,False,True,False,0.5800


### 5. Train-Test Split
We split the data into training and testing sets to evaluate our models. 80% is used for training, 20% for testing.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 6. Baseline Model Training & Evaluation
We'll train two powerful gradient boosting models, XGBoost and LightGBM, as our base learners. We train them on the unscaled data, as tree models are generally not sensitive to feature scaling.

#### 6.1. Model 1: XGBoost (Baseline)

In [30]:
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.10000110001,
    max_depth=7,
    random_state=42,
    n_jobs=-1
)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

xgb_model.fit(X_train, y_train)

predictions_model1 = xgb_model.predict(X_test)
clipped_predictions = np.clip(predictions_model1, 0, 1)

print(r2_score(y_test, clipped_predictions))
print(mean_absolute_error(y_test, clipped_predictions))

0.8857417642357159
0.04359361840632821


#### 6.2. Model 2: LightGBM (Baseline)

In [31]:
lgb_model = lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    n_estimators=100,
    learning_rate=0.099,
    num_leaves=141,
    random_state=42,
    n_jobs=-1
)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

lgb_model.fit(X_train, y_train)
predictions_model2 = lgb_model.predict(X_test)
clipped_predictions = np.clip(predictions_model2, 0, 1)


print(r2_score(y_test, clipped_predictions))
print(mean_absolute_error(y_test, clipped_predictions))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015559 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 415
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 19
[LightGBM] [Info] Start training from score 0.352605
0.8857480750905788
0.043579307508632315


### 7. Ensembling Experiment 1: Simple Averaging
A simple yet powerful ensembling technique is to average the predictions of our two base models. We clip the predictions to the valid [0, 1] range first.

In [21]:
clipped_preds1 = np.clip(predictions_model1, 0, 1)
clipped_preds2 = np.clip(predictions_model2, 0, 1)

# Calculate the average prediction
average_predictions = (clipped_preds1 + clipped_preds2) / 2.0

# Evaluate the ensembled predictions
ensembled_r2 = r2_score(y_test, average_predictions)

print(f"Model 1 R²: {r2_score(y_test, clipped_preds1):.5f}")
print(f"Model 2 R²: {r2_score(y_test, clipped_preds2):.5f}")
# print(f"Model 3 R²: {r2_score(y_test, clipped_preds3):.5f}")
print(f"Ensembled (Average) R²: {ensembled_r2:.5f}")

Model 1 R²: 0.88574
Model 2 R²: 0.88575
Ensembled (Average) R²: 0.88583


**Observation:** The simple averaging ensemble often provides a more robust score, sometimes outperforming the individual models.

### 8. Ensembling Experiment 2: Stacking
A more advanced technique is stacking, where a "meta-model" learns to combine the predictions of the base models. We use `cross_val_predict` to generate "out-of-fold" predictions for the meta-model's training set, which prevents data leakage.

In [22]:
# Train Base Models
xgb_model.fit(X_train, y_train)
lgb_model.fit(X_train, y_train)

# Generate Predictions for Meta-Model Training
oof_preds_xgb = cross_val_predict(xgb_model, X_train, y_train, cv=5, n_jobs=-1)
oof_preds_lgb = cross_val_predict(lgb_model, X_train, y_train, cv=5, n_jobs=-1)

# Stack these predictions horizontally to create the meta-model's training features
X_meta_train = np.column_stack((oof_preds_xgb, oof_preds_lgb))
y_meta_train = y_train # Use original training targets

# Train the Meta-Model
meta_model = xgb.XGBRegressor(n_estimators=50, learning_rate=0.1, max_depth=3, random_state=42)
meta_model.fit(X_meta_train, y_meta_train)

# Generate Predictions for Meta-Model Testing
test_preds_xgb = xgb_model.predict(X_test)
test_preds_lgb = lgb_model.predict(X_test)

# Stack these predictions horizontally
X_meta_test = np.column_stack((test_preds_xgb, test_preds_lgb))

# Make Final Predictions
final_predictions = meta_model.predict(X_meta_test)

# Evaluate
clipped_final_predictions = np.clip(final_predictions, 0, 1)
final_r2 = r2_score(y_test, clipped_final_predictions)

print(f"\nStacked Model R²: {final_r2:.5f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039840 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 415
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 19
[LightGBM] [Info] Start training from score 0.352605

Stacked Model R²: 0.88557


**Observation:** In this case, the simple average (88.58%) outperformed the stacking model (88.55%). This suggests our base models are highly correlated, and a simple average is the most effective way to combine them.

### 9. Hyperparameter Optimization (Bayesian Optimization)
To ensure our LightGBM model is optimally tuned, we use Bayesian Optimization. This method intelligently searches the hyperparameter space to find the combination that maximizes the cross-validated R² score.

In [23]:
from sklearn.model_selection import cross_val_score

# Define the function to optimize
def lgb_objective(max_depth, learning_rate, n_estimators, subsample, colsample_bytree, num_leaves):
    """Objective function for Bayesian Optimization"""

    # Ensure integer parameters are integers
    max_depth = int(max_depth) if max_depth > 0 else -1
    n_estimators = int(n_estimators)
    num_leaves = int(num_leaves)

    # Define the XGBoost model with current hyperparameters
    model = lgb.LGBMRegressor(
        objective='regression',
        metric='rmse',
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        num_leaves=num_leaves,
        random_state=42,
        n_jobs=-1
    )
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='r2').mean()

    return score

In [24]:
# Define the bounds for the hyperparameters
pbounds = {
    'max_depth': (3, 10),           # Search depths between 3 and 10
    'learning_rate': (0.01, 0.3),   # Search learning rates between 0.01 and 0.3
    'n_estimators': (100, 1000),    # Search number of estimators between 100 and 1000
    'subsample': (0.6, 1.0),        # Search subsample ratio between 0.6 and 1.0
    'colsample_bytree': (0.6, 1.0),  # Search feature fraction between 0.6 and 1.0
    'num_leaves': (10, 200)
}

In [25]:
from bayes_opt import BayesianOptimization

# Create the Bayesian Optimization object
optimizer = BayesianOptimization(
    f=lgb_objective,  # Function to optimize
    pbounds=pbounds,  # Parameter bounds
    random_state=42, # For reproducibility
    verbose=2        # Prints details of optimization steps (2 = print all, 1 = print improvements, 0 = silent)
)

# Run the optimization
# init_points: How many random points to explore initially
# n_iter: How many Bayesian optimization steps to perform afterwards
print("Starting Bayesian Optimization...")
optimizer.maximize(
    init_points=5,
    n_iter=25
)
print("Optimization complete.")

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [26]:
# Get the best parameters found
best_params = optimizer.max['params']

best_params['max_depth'] = int(best_params['max_depth'])
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['num_leaves'] = int(best_params['num_leaves'])


print("\nBest Hyperparameters Found:")
print(best_params)


Best Hyperparameters Found:
{'max_depth': 7, 'learning_rate': np.float64(0.0470015633697951), 'n_estimators': 264, 'subsample': np.float64(0.8322855548642619), 'colsample_bytree': np.float64(0.7851468884701903), 'num_leaves': 109}


**Observation:** The Bayesian Optimization found parameters that resulted in a cross-validated R² of ~88.6%. This confirms that our initial manual parameters were already very strong and close to the optimal performance ceiling for this feature set.

### 10. Final Submission
Based on our experiments, the best-performing and most robust strategy was the simple average ensemble of our tuned XGBoost and LightGBM models. The code below will train these models on the **full training dataset** and generate the `submission.csv` file.

In [27]:
# Load and Preprocess FULL Training Data
df_train = pd.read_csv('data/train.csv')

# Apply one-hot encoding
df_train_processed = pd.get_dummies(df_train, columns=['lighting', 'weather', 'time_of_day', 'road_type'])

# Add interaction feature (handle potential division by zero if num_lanes can be 0)
df_train_processed['curvature_lane_interaction'] = df_train_processed['curvature'] / df_train_processed['num_lanes']

# Define features (X_train_full) and target (y_train_full)
features_to_drop = ['accident_risk', 'id', 'road_signs_present', 'school_season']
X_train_full = df_train_processed.drop(columns=features_to_drop)
y_train_full = df_train_processed['accident_risk']
train_columns = X_train_full.columns
print(f"Training data preprocessed. Shape: {X_train_full.shape}")

# Train the XGBoost Model on FULL Data
print("Training XGBoost model...")
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=7,
    random_state=42,
    n_jobs=-1
)
xgb_model.fit(X_train_full, y_train_full)
print("XGBoost model trained.")

# Step 3: Train the LightGBM Model on FULL Data
print("Training LightGBM model...")
lgb_model = lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    n_estimators=100,
    learning_rate=0.1,
    num_leaves=141,
    random_state=42,
    n_jobs=-1
)
lgb_model.fit(X_train_full, y_train_full)
print("LightGBM model trained.")

# Load and Preprocess Test Data
print("Loading and preprocessing test data...")
df_test = pd.read_csv('data/test.csv')
test_ids = df_test['id'] # Store IDs for submission

# Apply the same one-hot encoding
df_test_processed = pd.get_dummies(df_test, columns=['lighting', 'weather', 'time_of_day', 'road_type'])

# Add the same interaction feature
df_test_processed['curvature_lane_interaction'] = df_test_processed['curvature'] / df_test_processed['num_lanes']

# Drop the id column AFTER using it, keep features consistent with X_train_full
df_test_processed = df_test_processed.drop(columns=['id', 'road_signs_present', 'school_season'])

# Align Test Columns EXACTLY with Training Columns
print("Aligning test data columns...")
X_test_final = df_test_processed.reindex(columns=train_columns, fill_value=0)
print(f"Test data preprocessed and aligned. Shape: {X_test_final.shape}")

# Make Predictions with Both Models
print("Making predictions...")
predictions_xgb = xgb_model.predict(X_test_final)
predictions_lgb = lgb_model.predict(X_test_final)

# Clip Predictions
clipped_preds_xgb = np.clip(predictions_xgb, 0, 1)
clipped_preds_lgb = np.clip(predictions_lgb, 0, 1)

# Average Predictions
average_predictions = (clipped_preds_xgb + clipped_preds_lgb) / 2.0
print("Predictions averaged.")

# Create Submission File
submission_df = pd.DataFrame({'id': test_ids, 'accident_risk': average_predictions})

# Save Submission File
submission_df.to_csv('submission.csv', index=False)

print("\ Submission file 'submission.csv' created successfully!")
print(submission_df.head())

<>:81: SyntaxWarning: invalid escape sequence '\ '
<>:81: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipython-input-3599393702.py:81: SyntaxWarning: invalid escape sequence '\ '
  print("\ Submission file 'submission.csv' created successfully!")


Training data preprocessed. Shape: (517754, 19)
Training XGBoost model...
XGBoost model trained.
Training LightGBM model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.043229 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 411
[LightGBM] [Info] Number of data points in the train set: 517754, number of used features: 19
[LightGBM] [Info] Start training from score 0.352377
LightGBM model trained.
Loading and preprocessing test data...
Aligning test data columns...
Test data preprocessed and aligned. Shape: (172585, 19)
Making predictions...
Predictions averaged.
\ Submission file 'submission.csv' created successfully!
       id  accident_risk
0  517754       0.293271
1  517755       0.122918
2  517756       0.183189
3  517757       0.312685
4  517758       0.407947


### 11. Notes
Several other libraries were also tested but eventually not used due to low score

In [28]:
# model = CatBoostRegressor(
#     random_state=42,
#     learning_rate=0.1,     # A common starting learning rate
#     depth=7,           # Start with relatively shallow trees
#     iterations=652
# )

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# model.fit(X_train_scaled, y_train)

# predictions_model3 = model.predict(X_test_scaled)
# clipped_predictions3 = np.clip(predictions_model3, 0, 1)

# r2_score(y_test, clipped_predictions3)

In [29]:
# model = RandomForestRegressor(n_estimators=100, random_state=42)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# model.fit(X_train_scaled, y_train)

# predictions = model.predict(X_test_scaled)
# clipped_predictions = np.clip(predictions, 0, 1)

# r2_score(y_test, clipped_predictions)